In [1]:
import torch
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import importlib
import time
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import scipy.io
import matplotlib.pyplot as plt
from functools import partial
import time
import sys
import os
import gc
import importlib
import RFrtModel_SingleFracture_SolverDA as RF_1Frac_DA

In [3]:
importlib.reload(RF_1Frac_DA)

<module 'RFrtModel_SingleFracture_SolverDF' from '/Users/huynh/Desktop/Python/UnitedFilter_SingleRFrt/RFrtModel_SingleFracture_SolverDF.py'>

## Reverse SDE:

In [5]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, n_dim, select_set, x0, time_steps, C, score_likelihood=None, \
                drift_fun=f, diffuse_fun=g, alpha_fun=cond_alpha, sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = torch.randn(ensemble_size, n_dim, dtype=torch.float64, device=device)
    t = 1.0

    # define storage
    if save_path:
        path_all = [xt]
        t_vec = [t]

    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
            xt += -dt*( drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-diffuse**2*score_likelihood(xt, t, C)) \
                  +np.sqrt(dt)*diffuse*torch.randn_like(xt)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*torch.randn_like(xt)
        
        xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt)
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

## Resampling for Direct Filter

In [7]:
def systematic_resample_pytorch(weights):
    N = len(weights)
    
    # Generate positions with a consistent random offset
    positions = (torch.rand(1) + torch.arange(N)) / N

    # Compute cumulative sum of weights
    cumulative_sum = torch.cumsum(weights, dim=0)

    # Initialize the indexes tensor
    indexes = torch.zeros(N, dtype=torch.long)
    
    # Initialize pointers
    i, j = 0, 0
    while i < N and j < len(cumulative_sum):
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    
    return indexes

## Run this block to obtain the reference solution

In [9]:
xt = 0
xe = 2
yt = 0
ye = 1
nx = 80
ny = 40
node_frt = ny + 1
device = 'cpu'
# device = 'cuda'
nt = 800
T = 5
t0 = 0
SDE_sigma = 0.005

h = (ye - yt) / ny
dt = (T - t0) / nt

frt_length = 1e-03
diff_frt = 2/ frt_length

timegrid = torch.linspace(0, T, nt + 1)
poros_subdom = 1
poros_gamma = 1 / frt_length
poros_frt = frt_length * poros_gamma

# Mesh generation and diffusion parameters
TriMesh_Subdom = RF_1Frac_DA.MeshGenerator_PureDiff(xt, xe, yt, ye, nx, ny, device)
n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1
DOFs_Local = [None, None]
NumEgs = [None, None]
NumEms = [None, None]
for k in range(2):
    DOFs_Local[k] = TriMesh_Subdom[k].DOFs
    NumEgs[k] = TriMesh_Subdom[k].NumEgs
    NumEms[k] = TriMesh_Subdom[k].NumEms
# print(TriMesh_Subdom[0].DirichletEgs.shape)
#diffusion = [torch.zeros(TriMesh_Subdom[k].NumEms) for k in range(2)]
DOFs = DOFs_Local[0]+DOFs_Local[1]+node_frt+ny
diff_1subdom = 1
diff_2subdom = 1
alpha_frt = diff_frt * frt_length

diffusion_ref = RF_1Frac_DA.compute_diffusion_single(diff_1subdom, diff_2subdom, TriMesh_Subdom, device)

GAUSSQUAD = RF_1Frac_DA.SetGaussQuad(2, 4, 3, device)
Eqn_Subdom = [RF_1Frac_DA.EqnBC(), RF_1Frac_DA.EqnBC()]
EqnBC_fracture = RF_1Frac_DA.EqnBC_frt()
Local_Mat = [None]*2
Local_Mat_pertb = [None]*2

for k in range(2):
    Local_Mat[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat(TriMesh_Subdom[k], dt, diffusion_ref[k], poros_subdom, device)
    
MatB = [None]*2
MatC = [None]*2

MatATest = [None]*2
Local_MatTest = [None]*2

for k in range(2):
    MatB[k], MatC[k] = RF_1Frac_DA.ComputeBandCblocks(TriMesh_Subdom[k], dt, poros_subdom,  device)
    
for k in range(2):
    MatATest[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dt, diffusion_ref[k], device)
    Local_MatTest[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatATest[k], MatB[k], MatC[k], dt)    

Glob_MatTest = RF_1Frac_DA.Assemble_StiffMax_PureDiff_Alpha_frt(TriMesh_Subdom, h, node_frt, ny, \
                                                                Local_MatTest, alpha_frt, dt, poros_frt, device)
BCs_frt = 3
BCs_Subdom = 3
X0 = torch.zeros((1, TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1), dtype=torch.float64, device=device)

X0 = X0.to(device)
state_target = X0

# state_ref = torch.zeros(nt+1, n_dim, device = device)
# state_ref[[0], :] += state_target

# for i in range(nt):
#     print(i)
#     noise2 = np.sqrt(dt)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, 1),\
#                                                                 dtype=torch.float64, device=device))
#     stateT = torch.transpose(state_ref[[i], :], 0, 1)
#     sln = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA(stateT, ny, node_frt, TriMesh_Subdom, nt, dt, \
#                                                            poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
#                                                            Glob_MatTest, GAUSSQUAD, Eqn_Subdom, \
#                                                            EqnBC_fracture, SDE_sigma, device, noise2, 0)
#     state_ref[[i+1], :] += torch.transpose(sln, 0, 1)

In [ ]:
state_ref_save = state_ref.clone()
state_ref_save = state_ref_save.numpy()

scipy.io.savemat('TestUnitedSF_RefSol_PureDiff_v1.mat', {'Est_State':state_ref_save})

## Setting for the United Filter algorithm

In [17]:
data = scipy.io.loadmat('TestUnitedSF_RefSol_PureDiff_v1.mat') ## Replace by the name of .mat file
state_ref = torch.from_numpy(data['ExactState'])

# data = scipy.io.loadmat('SpatialSelection.mat')
# ida_sub = torch.from_numpy(data['ida_sub_save'])
# filtering_steps = nt
n_dim = TriMesh_Subdom[0].DOFs+TriMesh_Subdom[1].DOFs+2*ny+1

ntEnSF = 50
filtering_steps = ntEnSF
timeTrue = torch.linspace(0, 1, nt+1)
tEnSF = torch.linspace(0, 1, filtering_steps+1)
indices = torch.searchsorted(timeTrue, tEnSF, right=False)

state_EnSF = state_ref[indices, :].clone() # extract temporal location

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.2
####################################################################
# EnSF setup
# define the diffusion process
eps_alpha = 0.05

# ensemble size
ensemble_size = 200
ensemble_true = 1
# forward Euler step
euler_steps = 200
sub_steps = 4

time_euler = torch.linspace(0, 1, euler_steps+1)
time_sub = torch.linspace(0, 1, sub_steps+1)
select_set = torch.searchsorted(time_euler, time_sub, right=False)

print(select_set)
# device = 'cpu'
# device = 'cuda'

def g_tau(t):
    return 1-t

# saving file name
exp_name = 'JointStatePara_RFrtModel_DecompDF_100Linear'

x_state = 5*torch.randn(ensemble_size, n_dim, device=device)
# x_state = x_state.to(torch.float16)
mem_state = state_target.element_size() * state_target.nelement()/1e+6
mem_ensemble = mem_state * ensemble_size
print(f'single state memory: {mem_state:.2f} MB')
print(f'state ensemble memory: {mem_ensemble:.2f} MB')

rmse_all = []
obs_save = []
est_save = torch.zeros(filtering_steps+1, n_dim, device = device)
est_save[[0], :] += torch.mean(x_state,dim=0)

x0filter = x_state

theta0 = torch.tensor([8, 8, 8], dtype=torch.float64, device =device); #Initial guess for the direct filter
Cov = torch.eye(3, dtype=torch.float64, device =device) * torch.tensor([4, 4, 4], dtype=torch.float64, device =device)

n = 30;  # Number of particles considered     
l= 3 # Number of United Filter's iterations
particles =torch.distributions.MultivariateNormal(theta0, Cov).sample((n,)) # initial particles
xs = [theta0]
weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
IntmediaPara = torch.sum(particles * weights.unsqueeze(1), dim=0) 
# SlnRefT = torch.transpose(ExtractSlnRef, 0, 1)
X = torch.zeros((ntEnSF+1, DOFs), dtype=torch.float64, device=device) #545 is the dof in the mesh considered
X[0, :] += state_EnSF[0, :]
for i in range(1, ntEnSF+1):
    X[i, :] += state_EnSF[i, :]
    
# Create the new vector B

length = state_EnSF.shape[1]
num_indices = int(0.75* length) #75% observations
# num_indices = int(1* length) # 100% observations


### Choosing spatial locations 
spa_indices = torch.randperm(length)[:num_indices]  # Random permutation, then take the first num_indices
spa_indices, _ = spa_indices.sort() 

### Extracting spatial locations
state_United = state_EnSF[:, spa_indices].clone()

state_United = state_United.to(torch.float64)
factorR = 0.2
R = factorR*torch.eye(DOFs, dtype=torch.float64, device=device)   # Covariance considered 
R_scaled = factor * R
Z = 0.2*torch.randn(num_indices, ntEnSF+1, dtype=torch.float64, device=device)
state_United += torch.transpose(Z, 0, 1)

NeumannEgs = [None]*2
for k in range(2):
    NeumannEgs[k] = TriMesh_Subdom[k].NeumannEgs

LocalMat_DF = [None]*2

MatD = RF_1Frac_DA.ComputeDblocks(TriMesh_Subdom, h, dtEnSF, ny, device)

tensor([  0,  50, 100, 150, 200])
single state memory: 0.13 MB
state ensemble memory: 25.99 MB
16241
12180


## Run this block for the iterative algorithm

In [ ]:
for i in range(filtering_steps):
    weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
    print(f'step={i}:')
    t1 = time.time()
    obs = state_United[[i+1], :].clone()+torch.randn_like(state_United[[i+1], :])*obs_sigma
    def score_likelihood(xt, t, C):
        # obs: (d)
        # xt: (ensemble, d)
        score_x = -(xt-obs)/obs_sigma**2
        tau = g_tau(t)
        return tau*score_x/C
    
    x0EnSF = x_state.clone()
    x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
    for j in range(l):
        weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
        IntmediaPara_First = IntmediaPara.clone()
        ##### EnSF 
        Local_Matloop = [None]*2
        MatAEnSF = [None]*2
        
        diffusion_loopEnSF = RF_1Frac_DA.compute_diffusion_single(IntmediaPara_First[0], IntmediaPara_First[1], \
                                                                  TriMesh_Subdom, device)
        for k in range(2):
            MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_loopEnSF[k], device)
            Local_Matloop[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)
        
        Glob_Mat_loop = RF_1Frac_DA.Assemble_StiffMax_PureDiff_Modify(TriMesh_Subdom, h, node_frt, ny, Local_Matloop, \
                                                                      MatD, IntmediaPara_First[2], dtEnSF, poros_frt,\
                                                                      device)
        
        noise1 = np.sqrt(dtEnSF)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, ensemble_size),\
                                                                        dtype=torch.float64, device=device))

        # x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
        sln_state = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA(x0filterT, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                     dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom,\
                                                                     Glob_Mat_loop, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                     SDE_sigma, device, noise1, 0)
        
        
        x_state = torch.transpose(sln_state, 0, 1)
        
        sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set,\
                              x0=x_state[:, spa_indices], time_steps=euler_steps,\
                              C=1, score_likelihood=score_likelihood)
        
        x_state[:, spa_indices] = sln_bar.clone()
    
        x_state[:, NumEgs[0] + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, NumEgs[0] + torch.arange(0, NumEms[0])],\
                        min=0, max = BCs_Subdom)

        x_state[:, DOFs_Local[0]+NumEgs[1]+torch.arange(0, NumEms[1])] =\
                    torch.clamp(x_state[:, DOFs_Local[0]+NumEgs[1]+torch.arange(0, NumEms[1])],\
                                min=0, max = BCs_Subdom)

        x_state[:, DOFs_Local[0] + DOFs_Local[1] + node_frt + torch.arange(0, ny)] =\
                    torch.clamp(x_state[:, DOFs_Local[0] + DOFs_Local[1] + node_frt + torch.arange(0, ny)],\
                                min=0, max = BCs_frt)
        
        x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min=-13, max=13)
        x_state[:, DOFs_Local[0]:DOFs_Local[0]+NumEgs[1]] = \
                    torch.clamp(x_state[:, DOFs_Local[0]:DOFs_Local[0]+NumEgs[1]], min=-13, max=13)
        x_state[:, DOFs_Local[0]+DOFs_Local[1]:DOFs_Local[0]+DOFs_Local[1]+ny+1] = \
                    torch.clamp(x_state[:, DOFs_Local[0]+DOFs_Local[1]:DOFs_Local[0]+DOFs_Local[1]+ny+1], min=0, max=13)
        
        x_state[:, NeumannEgs[0]] = 0
        x_state[:, DOFs_Local[0]+NeumannEgs[1]] = 0


        ####     
        xstate_bar = torch.mean(x_state,dim=0) # \bar{X}^{(l)}_{i+1}
       
        if i <= 2*int(filtering_steps // 5):
            Ep = torch.diag(torch.tensor([0.81, 0.81, 0.81], dtype=torch.float64))  
        elif i < 4*int(filtering_steps // 5) and i > 2*int(filtering_steps // 5):
            Ep = torch.diag(torch.tensor([0.49, 0.49, 0.49], dtype=torch.float64))            
        else:
            Ep = torch.diag(torch.tensor([0.08, 0.08, 0.08], dtype=torch.float64))
            
        cholEp = torch.linalg.cholesky(Ep)
        particles += torch.randn(n, 3, dtype=torch.float64)@cholEp #\tilde{\gamma}^{k, (l+1)}_{i+1}
        
        ## avoid having negative permeability
        for qq in range(particles.shape[1]):  # Iterate over columns
            condition = particles[:, qq] < 0  # Check for negative values in the column
            if condition.any():  # If there are negative values
                print(qq)
                print(f"negative detected in {'d2' if qq == 1 else 'd1' if qq == 0 else 'd3'}")
                # Find the smallest positive entry in the column
                positive_values = particles[:, qq][particles[:, qq] > 0]
                if positive_values.numel() > 0:  # Ensure there are positive values
                    smallest_positive = positive_values.min()
                    # Replace negative values with the smallest positive entry
                    particles[:, qq] = torch.where(
                        condition,
                        smallest_positive,
                        particles[:, qq]
                    )

        z = xstate_bar.clone() # 
        P = torch.zeros(n, DOFs, dtype=torch.float64, device=device)

        x0DFT = torch.transpose(est_save[[i], :].clone(), 0, 1)
     
        for o, particle in enumerate(particles):            
            MatADF = [None]*2
            diff_DF = RF_1Frac_DA.compute_diffusion_single(particle[0], particle[1], TriMesh_Subdom, device)
  
            for k in range(2):
                MatADF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diff_DF[k], device)
                LocalMat_DF[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatADF[k], MatB[k], MatC[k], dtEnSF) 
            
            GlobMatDF = \
            RF_1Frac_DA.Assemble_StiffMax_PureDiff_Modify(TriMesh_Subdom, h, node_frt, ny, LocalMat_DF, MatD, \
                                                          particle[2], dtEnSF, poros_frt, device)
            
            SlnDF = \
            RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA(x0DFT, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                             dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom, \
                                                             GlobMatDF, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                             SDE_sigma, device, noise1, 0)
            SlnDFT = torch.transpose(SlnDF, 0, 1)
           
            P[o, :] += SlnDFT[-1, :] 

        P -=  z.unsqueeze(0)   # Broadcasting is already efficient

        # Step 2: Solve R * X = P^T for X
        Rsolved = P*(1/factorR)
        
        # Step 3: Compute norms
        Rnorm = torch.sum(Rsolved * P, dim=1)
        
        Rnorm_clone = Rnorm.clone() / 1
        
        ## Scale the norm to avoid too big values
        if torch.min(Rnorm_clone) > 1100:
            alp_factor = torch.min(Rnorm_clone) / 1100 
            Rnorm_clone /= alp_factor 
            

        # Step 4: Update weights
        weights.mul_(torch.exp(-0.5 * Rnorm_clone))  # In-place operation

        weights.add_(1e-300)  # Add directly without creating new tensors
        weights.div_(weights.sum())  # Normalize in-place
        
        # Step 5: Resampling and updating particles
        indexes = systematic_resample_pytorch(weights)
        
        particles[:] = particles[indexes]
        
        # Step 6: Update mean
        mu = torch.sum(particles * weights.unsqueeze(1), dim=0)
        IntmediaPara = mu.clone() 
    
    ## EnSF with \bar{\gamma}^{L}_{i+1}
    IntmediaPara_lastEnSF = IntmediaPara.clone()
    diffusion_ref_pertb = \
    RF_1Frac_DA.compute_diffusion_single(IntmediaPara_lastEnSF[0], IntmediaPara_lastEnSF[1],\
                                         TriMesh_Subdom, device)
    for k in range(2):
        MatAEnSF[k] = RF_1Frac_DA.ComputeAblock(TriMesh_Subdom[k], dtEnSF, diffusion_ref_pertb[k], device)
        Local_Mat_pertb[k] = RF_1Frac_DA.Darcy_MFEM_StiffMat_Modify(MatAEnSF[k], MatB[k], MatC[k], dtEnSF)
    
    Glob_Mat_pertb = RF_1Frac_DA.Assemble_StiffMax_PureDiff_Modify(TriMesh_Subdom, h, node_frt, ny, Local_Mat_pertb,\
                                                                   MatD, IntmediaPara_lastEnSF[2], dtEnSF,\
                                                                   poros_frt, device)

    noise1 = np.sqrt(dtEnSF)*SDE_sigma*torch.randn_like(torch.zeros((NumEms[0]+NumEms[1]+ny, ensemble_size),\
                                                                    dtype=torch.float64, device=device))

    x0filterT_last = torch.transpose(x0EnSF.clone(), 0, 1)
    sln_state = RF_1Frac_DA.ComputSol_DFFracture_Afrt_Ksubdom_DA(x0filterT_last, ny, node_frt, TriMesh_Subdom, ntEnSF, \
                                                                 dtEnSF, poros_subdom, poros_frt, BCs_frt, BCs_Subdom,\
                                                                 Glob_Mat_pertb, GAUSSQUAD, Eqn_Subdom, EqnBC_fracture,\
                                                                 SDE_sigma, device, noise1, 0)
    
    x_state = torch.transpose(sln_state, 0, 1)
    sln_bar = reverse_SDE(DOFs_Local, NumEgs, NumEms, NeumannEgs, num_indices, select_set,\
                              x0=x_state[:, spa_indices], time_steps=euler_steps,\
                              C=1, score_likelihood=score_likelihood)
    x_state[:, spa_indices] = sln_bar.clone()
    
    x_state[:, NumEgs[0] + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, NumEgs[0] + torch.arange(0, NumEms[0])],\
                        min=0, max = BCs_Subdom)

    x_state[:, DOFs_Local[0]+NumEgs[1]+torch.arange(0, NumEms[1])] =\
                torch.clamp(x_state[:, DOFs_Local[0]+NumEgs[1]+torch.arange(0, NumEms[1])],\
                            min=0, max = BCs_Subdom)

    x_state[:, DOFs_Local[0] + DOFs_Local[1] + node_frt + torch.arange(0, ny)] =\
                torch.clamp(x_state[:, DOFs_Local[0] + DOFs_Local[1] + node_frt + torch.arange(0, ny)],\
                            min=0, max = BCs_frt)
    
    x_state[:, :NumEgs[0]] = torch.clamp(x_state[:, :NumEgs[0]], min=-13, max=13)
    x_state[:, DOFs_Local[0]:DOFs_Local[0]+NumEgs[1]] = \
                torch.clamp(x_state[:, DOFs_Local[0]:DOFs_Local[0]+NumEgs[1]], min=-13, max=13)
    x_state[:, DOFs_Local[0]+DOFs_Local[1]:DOFs_Local[0]+DOFs_Local[1]+ny+1] = \
                torch.clamp(x_state[:, DOFs_Local[0]+DOFs_Local[1]:DOFs_Local[0]+DOFs_Local[1]+ny+1], min=0, max=13)
    
    x_state[:, NeumannEgs[0]] = 0
    x_state[:, DOFs_Local[0]+NeumannEgs[1]] = 0 
    
    x_statebarEnSF = torch.zeros_like(est_save[[i+1], :])
    x_statebarEnSF += torch.mean(x_state.clone(),dim=0) #\bar{X^{L}}_{i+1} = \bar{X}_{i+1}

    est_save[[i+1], :] += torch.mean(x_state.clone(),dim=0)
    xs.append(mu)
    diff = torch.sqrt(torch.mean((est_save[[i+1], :]  - x_statebarEnSF)**2)).item()
    rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
    rmse = torch.sqrt(torch.mean((x_statebarEnSF  - state_EnSF[[i+1], :])**2)).item()
    
    # get time
    if x_state.device.type == 'cuda':
        torch.cuda.current_stream().synchronize()
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2-t1:.4f} ')

    # save information
    rmse_all.append(rmse_temp)
    # break
    if rmse_temp > 1000:
        print('diverge!')
        break
        
state_savepy  = state_ref.clone()
est_savepy = est_save.clone()

state_savepy = state_savepy.cpu().numpy()
est_savepy = est_savepy.cpu().numpy()
rmse_all = np.array(rmse_all)

# scipy.io.savemat('UnitedFilter_2DRft_3Para_100LinearPartialOb_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})

# scipy.io.savemat('UnitedFilter_2DRft_3Para_75LinearPartialOb_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})

## Run the results and plot the figures in MATLAB